# Práctico 4: Generación de Variables Aleatorias Discretas

## Algoritmos

In [1]:
import random as rnd
import numpy as np
import time

### Generación de VAs Discretas con sus probabilidades de masa 

In [2]:
def discrete_distribution(p: list[float], x: list[int], sz: int = 1) -> list[int]:
    """
    Generate a random number from a discrete distribution.

    Parameters:
    p: list[float] - List of probabilities.
    x: list[int] - List of values.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(p: list[float], x: list[int]) -> int:
        r = rnd.random()
        i = 0; prob = p[i]
        while r >= prob: i += 1; prob += p[i]
        return x[i]
    
    assert len(p) == len(x), "The probabilities and values lists must have the same length."
    return [generate(p, x) for _ in range(sz)]

def discrete_distribution_function(p: callable, sz: int = 1) -> list[int]:
    """
    Generate a random number from a discrete distribution.
    This considers the values from 0 to inf.

    Parameters:
    p: callable - Probability function.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(p: callable) -> int:
        r = rnd.random(); i = 0
        prob = p(i)
        while r >= prob: i += 1; prob += p(i)
        return i
    
    return [generate(p) for _ in range(sz)]

if __name__ == "__main__":
    p = [0.1, 0.2, 0.3, 0.4]
    x = [1, 2, 3, 4]
    c = [0]*len(x)
    for i in discrete_distribution(p, x, 10**6): c[i-1] += 1
    print([c[i]/sum(c) for i in range(len(c))])

    p = lambda x: x/10
    c = [0]*5
    for i in discrete_distribution_function(p, 10**6): c[i] += 1
    print([c[i]/sum(c) for i in range(len(c))])

[0.100004, 0.200823, 0.299931, 0.399242]
[0.0, 0.100382, 0.199998, 0.300357, 0.399263]


### Generación de Uniforme Discreta

In [3]:
def uniform_discrete_distribution(a: int, b: int, sz: int = 1) -> list[int]:
    """
    Generate a random number from a uniform discrete distribution.

    Parameters:
    a: int - Lower bound.
    b: int - Upper bound.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(a: int, b: int) -> int:
        U = rnd.random()
        return a + int((b - a + 1) * U)
    
    return [generate(a, b) for _ in range(sz)]

if __name__ == "__main__":
    a, b = 1, 6
    c = [0]*(b-a+1)
    for i in uniform_discrete_distribution(a, b, 10**6): c[i-a] += 1
    print([c[i]/sum(c) for i in range(len(c))])

[0.166628, 0.166871, 0.166457, 0.16659, 0.166984, 0.16647]


#### Generación de una permutación aleatoria

In [4]:
def random_permutation(a: list) -> list:
    """
    Generate a random permutation of a list.

    Parameters:
    a: list - List to be permuted.

    Returns:
    list - Random permutation of the list.
    """
    b = a.copy()
    n = len(b)
    for j in range(n-1, 0, -1):
        i = uniform_discrete_distribution(0, j)[0]
        b[i], b[j] = b[j], b[i]
    return b

def random_sample(a: list, k: int, inv: bool = False) -> list:
    """
    Generate a random sample of k elements from a list.

    Parameters:
    a: list - List to be sampled.
    k: int - Number of elements to be sampled.
    inv: bool - If True, the sample will be the complement of the random sample.

    Returns:
    list - Random sample of k elements from the list.
    """
    n = len(a)
    if k > n/2: return random_sample(a, n-k, True)

    b = a.copy()
    for j in range(n-1, n-k-1, -1):
        i = uniform_discrete_distribution(0, j)[0]
        b[i], b[j] = b[j], b[i]
    return b[:n-k] if inv else b[n-k:]

if __name__ == "__main__":
    a = [1, 2, 3, 4, 5]
    print(random_permutation(a))
    for i in range(6):
        print(random_sample(a, i))

[2, 1, 4, 3, 5]
[]
[3]
[4, 2]
[4, 2, 3]
[1, 2, 5, 4]
[1, 2, 3, 4, 5]


#### Cálculo de sumatoria de valores de una función

In [5]:
def monte_carlo_sum(f: callable, n: int, m: int) -> float:
    """
    Estimate the summatory of a function using the Monte Carlo method.

    Parameters:
    f: callable - Function to estimate the summatory.
    n: int - Number of samples.
    m: int - Number of iterations.

    Returns:
    float - Estimated summatory of the function.
    """
    mean = 0
    for r in uniform_discrete_distribution(1, n, m):
        mean += f(r)
    return mean / m * n

if __name__ == "__main__":
    f = lambda x: np.exp(1/x)
    n = 10_000
    m = 100
    print(monte_carlo_sum(f, n, m))

10007.525587994402


### Generación de una VA Geométrica

In [6]:
def geometric_distribution(p: float, sz: int = 1) -> list[int]:
    """
    Generate a random number from a geometric distribution.

    Parameters:
    p: float - Probability of success.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(p: float) -> int:
        U = rnd.random()
        return int(np.log(1 - U) / np.log(1 - p)) + 1

    return [generate(p) for _ in range(sz)]

if __name__ == "__main__":
    p = 0.3
    c = dict()
    for x in geometric_distribution(p, 10**6): c[x] = c.get(x, 0) + 1
    for k in sorted(c.keys()): 
        if k > 20: break
        print(k, c[k]/sum(c.values()))

1 0.299148
2 0.21041
3 0.146247
4 0.103266
5 0.072175
6 0.050872
7 0.03539
8 0.024702
9 0.017384
10 0.012111
11 0.008355
12 0.006059
13 0.004126
14 0.002934
15 0.002028
16 0.001455
17 0.001023
18 0.000702
19 0.000501
20 0.000312


### Generación de una VA Bernoulli

In [7]:
def bernoulli_distribution(p: float, sz: int = 1) -> list[int]:
    """
    Generate a random number from a Bernoulli distribution.

    Parameters:
    p: float - Probability of success.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    if sz == 1:
        return [1 if rnd.random() < p else 0]
    
    # Using the geometric distribution
    r_list = [0] * sz
    j = geometric_distribution(p)[0] - 1
    while j < sz: r_list[j] = 1; j += geometric_distribution(p)[0]
    return r_list

if __name__ == "__main__":
    p = 0.3
    c = dict()
    for x in bernoulli_distribution(p, 10**6): c[x] = c.get(x, 0) + 1
    print(c[0]/sum(c.values()), c[1]/sum(c.values()))

    c.clear()
    for _ in range(10**6):
        x = bernoulli_distribution(p)[0]
        c[x] = c.get(x, 0) + 1
    print(c[0]/sum(c.values()), c[1]/sum(c.values()))

0.700597 0.299403
0.699948 0.300052


### Generación de una VA Poisson

In [8]:
def poisson_distribution(l: float, sz: int = 1) -> list[int]:
    """
    Generate a random number from a Poisson distribution.

    Parameters:
    l: float - Rate of the Poisson distribution.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(l: float) -> int:
        p = np.exp(-l); F = p
        U = rnd.random()

        for j in range(1, int(l) + 1): p *= l / j; F += p
        if U >= F:
            j = int(l) + 1
            while U >= F: p *= l / j; F += p; j += 1
            return j - 1
        else:
            j = int(l)
            while U < F: F -= p; p *= j / l; j -= 1
            return j + 1

    return [generate(l) for _ in range(sz)]

if __name__ == "__main__":
    l = 3
    c = dict()
    for x in poisson_distribution(l, 10**6): c[x] = c.get(x, 0) + 1
    for k in sorted(c.keys()): print(k, c[k]/sum(c.values()))

0 0.049841
1 0.148983
2 0.224062
3 0.224329
4 0.168266
5 0.100651
6 0.050295
7 0.021617
8 0.008101
9 0.002758
10 0.000794
11 0.000231
12 5.7e-05
13 1e-05
14 3e-06
15 2e-06


### Generación de una VA Binomial

In [9]:
def binomial_distribution(n: int, p: float, sz: int = 1) -> list[int]:
    """
    Generate a random number from a binomial distribution.

    Parameters:
    n: int - Number of trials.
    p: float - Probability of success.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(n: int, p: float) -> int:
        c = p / (1 - p); prob = (1 - p)**n
        F = prob; U = rnd.random()
        
        for j in range(1, int(n * p) + 1): prob *= c * (n - j + 1) / j; F += prob
        if U >= F:
            j = int(n * p) + 1
            while U >= F: prob *= c * (n - j + 1) / j; F += prob; j += 1
            return j - 1
        else:
            j = int(n * p)
            while U < F: F -= prob; prob *= j / (c * (n - j + 1)); j -= 1
            return j + 1

    if p > 0.5: return [n - generate(n, 1 - p) for _ in range(sz)]
    else: return [generate(n, p) for _ in range(sz)]

if __name__ == "__main__":
    n, p = 10, 0.3
    c = dict()
    for x in binomial_distribution(n, p, 10**6): c[x] = c.get(x, 0) + 1
    for k in sorted(c.keys()): print(k, c[k]/sum(c.values()))

0 0.028311
1 0.121085
2 0.233673
3 0.266063
4 0.199975
5 0.103183
6 0.037115
7 0.009018
8 0.001435
9 0.000132
10 1e-05


### Método de aceptación y rechazo

In [10]:
def rejection_method(Y: callable, p: callable, q: callable, c: float, sz: int = 1) -> list[int]:
    """
    Estimate the expected value of a function using the rejection method.

    Parameters:
    Y: callable - Function to simulate the random variable.
    p: callable - Probability function of the random variable of X.
    q: callable - Probability function of the random variable of Y.
    c: float - Constant such that p(x) <= c * q(x) for all x.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(Y: callable, p: callable, q: callable, c: float) -> int:
        while True:
            x = Y()
            U = rnd.random()
            if U < p(x) / (c * q(x)): return x
    
    return [generate(Y, p, q, c) for _ in range(sz)]

### Método de la urna

In [11]:
def urn_method(n: int, p: list[float], x: list[int], sz: int = 1) -> list[int]:
    """
    Generate a random number from an urn method.

    Parameters:
    n: int - Number of balls.
    p: list[float] - List of probabilities.
    x: list[int] - List of values.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def simulate(A: list[int]) -> int:
        return A[uniform_discrete_distribution(0, len(A) - 1)[0]]

    A = []
    for i in range(len(p)): A.extend([x[i]] * int(p[i] * n))
    return [simulate(A) for _ in range(sz)]

### Método de la tasa discreta de riesgo

In [12]:
def risk_rate_method(p: callable, sz: int = 1) -> list[int]:
    """
    Generate a random number from a risk rate method.

    Parameters:
    p: callable - Probability function.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(p: callable) -> int:
        prob = p(0); s = 0; i = 0
        while True:
            U = rnd.random()
            if U < prob / (1 - s): return i
            i += 1; s += prob; prob = p(i)

    return [generate(p) for _ in range(sz)]

## Ejercicio 1

In [13]:
def ej1_a(n, m, r):
    def win(r, n):
        v = random_permutation([i for i in range(1, n+1)])
        if all(v[i] == i+1 for i in range(r)): return [1, all(v[i] != i+1 for i in range(r, n))]
        return [0, 0]
    
    c = [0, 0]
    for _ in range(m):
        w = win(r, n)
        c[0] += w[0]
        c[1] += w[1]
    return [c[0]/m, c[1]/m]

def ej1_b(n, m):
    def simulate(n):
        v = random_permutation([i for i in range(1, n+1)])
        return sum(v[i] == i+1 for i in range(n))
    
    e = 0; v = 0
    for _ in range(m):
        x = simulate(n)
        e += x
        v += x**2
    e /= m; v /= m; v -= e**2
    return [e, v]

m_list = [10**i for i in range(2, 6)]
print("EJERCICIO 1A:")
for m in m_list:
    c = ej1_a(100, m, 10)
    print(f"    Para m = {m} tenemos que c_i = {c[0]}, c_ii = {c[1]}")

print("EJERCICIO 1B:")
for m in m_list:
    c = ej1_b(100, m)
    print(f"    Para m = {m} tenemos que E[X] = {c[0]}, V[X] = {c[1]}")

EJERCICIO 1A:
    Para m = 100 tenemos que c_i = 0.0, c_ii = 0.0
    Para m = 1000 tenemos que c_i = 0.0, c_ii = 0.0
    Para m = 10000 tenemos que c_i = 0.0, c_ii = 0.0
    Para m = 100000 tenemos que c_i = 0.0, c_ii = 0.0
EJERCICIO 1B:
    Para m = 100 tenemos que E[X] = 1.18, V[X] = 1.3276000000000003
    Para m = 1000 tenemos que E[X] = 0.947, V[X] = 0.862191
    Para m = 10000 tenemos que E[X] = 0.9872, V[X] = 0.99603616
    Para m = 100000 tenemos que E[X] = 0.99337, V[X] = 0.9977460430999999


## Ejercicio 2

In [14]:
n = 10**4
m = 100
g = lambda x : np.exp(x/n)

print("EJERCICIO 2A:")
v = monte_carlo_sum(g, n, m)
print(f"     La estimación por Monte Carlo de la sumatoria dada es: {monte_carlo_sum(g, n, m)}")

print("EJERCICIO 2B:")
v = sum(g(x) for x in range(1, m+1))
print(f"     La estimación directa de la sumatoria dada es: {v}")


EJERCICIO 2A:
     La estimación por Monte Carlo de la sumatoria dada es: 17175.806988029344
EJERCICIO 2B:
     La estimación directa de la sumatoria dada es: 100.50669600897406


## Ejercicio 3

In [15]:
def ej3_simulation():
    s = set()
    r = 0
    while len(s) < 11: s.add(sum(uniform_discrete_distribution(1, 6, 2))); r += 1
    return r

def ej3_a(n):
    e = 0; d = 0
    for _ in range(n):
        x = ej3_simulation()
        e += x
        d += x**2
    e /= n; d /= n; d = np.sqrt(d - e**2)
    return [e, d]

def ej3_b(n):
    p1 = 0; p2 = 0
    for _ in range(n):
        x = ej3_simulation()
        p1 += x >= 15
        p2 += x <= 9
    return [p1/n, p2/n]

n_list = [10**i for i in range(2, 6)]
print("EJERCICIO 3A:")
for n in n_list:
    c = ej3_a(n)
    print(f"    Para n = {n} tenemos que E[X] = {c[0]}, D[X] = {c[1]}")

print("EJERCICIO 3B:")
for n in n_list:
    c = ej3_b(n)
    print(f"    Para n = {n} tenemos que P(X >= 15) = {c[0]}, P(X <= 9) = {c[1]}")

EJERCICIO 3A:
    Para n = 100 tenemos que E[X] = 67.5, D[X] = 40.415219905377235
    Para n = 1000 tenemos que E[X] = 60.66, D[X] = 35.14860452421975
    Para n = 10000 tenemos que E[X] = 61.5476, D[X] = 36.174932401319005
    Para n = 100000 tenemos que E[X] = 61.31333, D[X] = 36.14298208935035
EJERCICIO 3B:
    Para n = 100 tenemos que P(X >= 15) = 1.0, P(X <= 9) = 0.0
    Para n = 1000 tenemos que P(X >= 15) = 1.0, P(X <= 9) = 0.0
    Para n = 10000 tenemos que P(X >= 15) = 0.9986, P(X <= 9) = 0.0
    Para n = 100000 tenemos que P(X >= 15) = 0.99862, P(X <= 9) = 0.0


## Ejercicio 4

In [16]:
def ej4_rechazo(n):
    def p(x): return [0.11, 0.14, 0.09, 0.08, 0.12, 0.10, 0.09, 0.07, 0.11, 0.09][x-1]
    def q(x): return 1/10
    def Y(): return uniform_discrete_distribution(1, 10)[0]
    c = 1.4

    vals = rejection_method(Y, p, q, c, n)

    # Check if the distribution is correct
    print("EJERCICIO 4 RECHAZO:")
    c = dict()
    for v in vals: c[v] = c.get(v, 0) + 1
    for k in sorted(c.keys()): print(f"{k}: {c[k]/sum(c.values())}", end=", ")
    print()

def ej4_transformada_inversa(n):
    x = [i for i in range(1, 11)]
    p = [0.11, 0.14, 0.09, 0.08, 0.12, 0.10, 0.09, 0.07, 0.11, 0.09]

    p, x = zip(*sorted(zip(p, x), reverse=True))
    vals = discrete_distribution(p, x, n)
    
    # Check if the distribution is correct
    print("EJERCICIO 4 TRANSFORMADA INVERSA:")
    c = dict()
    for v in vals: c[v] = c.get(v, 0) + 1
    for k in sorted(c.keys()): print(f"{k}: {c[k]/sum(c.values())}", end=", ")
    print()

def ej4_urna(n):
    def simulate(A):
        return A[uniform_discrete_distribution(1, len(A))[0]-1]

    x = [i for i in range(1, 11)]
    p = [0.11, 0.14, 0.09, 0.08, 0.12, 0.10, 0.09, 0.07, 0.11, 0.09]
    tam = 100

    vals = urn_method(tam, p, x, n)

    # Check if the distribution is correct
    print("EJERCICIO 4 URNA:")
    c = dict()
    for v in vals: c[v] = c.get(v, 0) + 1
    for k in sorted(c.keys()): print(f"{k}: {c[k]/sum(c.values())}", end=", ")
    print()

n_list = [10**4]
for n in n_list:
    act_time = time.time()
    ej4_rechazo(n)
    print(f"    Time: {time.time()-act_time}")

    act_time = time.time()
    ej4_transformada_inversa(n)
    print(f"    Time: {time.time()-act_time}")

    act_time = time.time()
    ej4_urna(n)
    print(f"    Time: {time.time()-act_time}")

EJERCICIO 4 RECHAZO:
1: 0.107, 2: 0.1387, 3: 0.0912, 4: 0.0798, 5: 0.1221, 6: 0.1041, 7: 0.09, 8: 0.0687, 9: 0.1087, 10: 0.0897, 
    Time: 0.01599907875061035
EJERCICIO 4 TRANSFORMADA INVERSA:
1: 0.1124, 2: 0.1392, 3: 0.0918, 4: 0.0728, 5: 0.1209, 6: 0.1043, 7: 0.0911, 8: 0.0705, 9: 0.1099, 10: 0.0871, 
    Time: 0.00400090217590332
EJERCICIO 4 URNA:
1: 0.1022, 2: 0.1461, 3: 0.0856, 4: 0.08, 5: 0.1251, 6: 0.1022, 7: 0.0885, 8: 0.0722, 9: 0.1061, 10: 0.092, 
    Time: 0.009999990463256836


## Ejercicio 5

In [17]:
def ej5_method1(n, p, m):
    vals = binomial_distribution(n, p, m)
    
    c = dict()
    for v in vals: c[v] = c.get(v, 0) + 1

    return [max(c, key=c.get), c.get(0, 0)/sum(c.values()), c.get(10, 0)/sum(c.values())]

def ej5_method2(n, p, m):
    vals = [sum(bernoulli_distribution(p, n)) for _ in range(m)]

    c = dict()
    for v in vals: c[v] = c.get(v, 0) + 1

    return [max(c, key=c.get), c.get(0, 0)/sum(c.values()), c.get(10, 0)/sum(c.values())]

m = 10**4; n = 10; p = 0.3
print("EJERCICIO 5A: Transformada inversa de la binomial")
act_time = time.time()
c = ej5_method1(n, p, m)
act_time = time.time() - act_time
print(f"    El valor máximo de la distribución binomial es: {c[0]}")
print(f"    La probabilidad de que X = 0 es: {c[1]}")
print(f"    La probabilidad de que X = 10 es: {c[2]}")
print(f"    Time: {act_time}")
print()

print("EJERCICIO 5B: N Bernoullis")
act_time = time.time()
c = ej5_method2(n, p, m)
act_time = time.time() - act_time
print(f"    El valor máximo de la distribución binomial es: {c[0]}")
print(f"    La probabilidad de que X = 0 es: {c[1]}")
print(f"    La probabilidad de que X = 10 es: {c[2]}")
print(f"    Time: {act_time}")

EJERCICIO 5A: Transformada inversa de la binomial
    El valor máximo de la distribución binomial es: 3
    La probabilidad de que X = 0 es: 0.0301
    La probabilidad de que X = 10 es: 0.0
    Time: 0.011998653411865234

EJERCICIO 5B: N Bernoullis
    El valor máximo de la distribución binomial es: 3
    La probabilidad de que X = 0 es: 0.0289
    La probabilidad de que X = 10 es: 0.0001
    Time: 0.0970001220703125


## Ejercicio 6

In [18]:
def ej6_method1(x, p, m):
    nw_p, nw_x = zip(*sorted(zip(p, x), reverse=True))
    vals = discrete_distribution(nw_p, nw_x, m)

    # Check if the distribution is correct
    print("EJERCICIO 6A:")
    c = dict()
    for v in vals: c[v] = c.get(v, 0) + 1
    for k in sorted(c.keys()): print(f"{k}: {c[k]/sum(c.values())}", end=", ")
    print()

def ej6_method2(x, p, m):
    q = [(1 - 0.45)**4]
    while len(q) < len(p):
        q.append(q[-1] * 0.45 / (1 - 0.45) * (4 - len(q) + 1) / len(q))

    def nw_p(x): return p[x]
    def nw_q(x): return q[x]
    def nw_Y(): return binomial_distribution(4, 0.45)[0]
    c = nw_p(3) / nw_q(4) + 1e-4

    vals = rejection_method(nw_Y, nw_p, nw_q, c, m)

    # Check if the distribution is correct
    print("EJERCICIO 6B:")
    c = dict()
    for v in vals: c[v] = c.get(v, 0) + 1
    for k in sorted(c.keys()): print(f"{k}: {c[k]/sum(c.values())}", end=", ")
    print()

x = [i for i in range(5)]
p = [0.15, 0.20, 0.10, 0.35, 0.20]
m = 10**4
act_time = time.time()
ej6_method1(x, p, m)
print(f"    Time: {time.time()-act_time}\n")

act_time = time.time()
ej6_method2(x, p, m)
print(f"    Time: {time.time()-act_time}\n")

EJERCICIO 6A:
0: 0.1525, 1: 0.1981, 2: 0.1009, 3: 0.3453, 4: 0.2032, 
    Time: 0.0030012130737304688

EJERCICIO 6B:
0: 0.1505, 1: 0.1975, 2: 0.1027, 3: 0.3512, 4: 0.1981, 
    Time: 0.15799760818481445



## Ejercicio 7

In [19]:
def poisson_distribution_not_optimized(l: float, sz: int = 1) -> list[int]:
    """
    Generate a random number from a Poisson distribution NOT optimized.

    Parameters:
    l: float - Rate of the Poisson distribution.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(l: float) -> int:
        p = np.exp(-l); F = p
        U = rnd.random(); i = 0
        
        while U >= F:
            i += 1; p *= l / i; F += p
        return i

    return [generate(l) for _ in range(sz)]

l = 10
n = 10**4
print(f"EJERCICIO 7: Generación de Poisson común")
vals = poisson_distribution_not_optimized(l, n)
p = sum(1 if v > 2 else 0 for v in vals) / len(vals)
print(f"    La probabilidad de que X > 2 es: {p}")

# Check if the normal Poisson distribution is correct
c = dict()
for v in vals: c[v] = c.get(v, 0) + 1
for k in sorted(c.keys()): 
    if k > 13: break
    print(f"{k}: {c[k]/sum(c.values())}", end=", ")
print("\n")

print(f"EJERCICIO 7: Generación de Poisson optimizada")
vals = poisson_distribution(l, n)
p = sum(1 if v > 2 else 0 for v in vals) / len(vals)
print(f"    La probabilidad de que X > 2 es: {p}")

# Check if the optimized Poisson distribution is correct
c = dict()
for v in vals: c[v] = c.get(v, 0) + 1
for k in sorted(c.keys()):
    if k > 13: break
    print(f"{k}: {c[k]/sum(c.values())}", end=", ")
print("\n")

EJERCICIO 7: Generación de Poisson común
    La probabilidad de que X > 2 es: 0.9972
1: 0.0007, 2: 0.0021, 3: 0.0088, 4: 0.0199, 5: 0.0357, 6: 0.0592, 7: 0.0891, 8: 0.1159, 9: 0.1289, 10: 0.1169, 11: 0.1157, 12: 0.0975, 13: 0.0695, 

EJERCICIO 7: Generación de Poisson optimizada
    La probabilidad de que X > 2 es: 0.9968
1: 0.0005, 2: 0.0027, 3: 0.0076, 4: 0.0209, 5: 0.0388, 6: 0.0637, 7: 0.0881, 8: 0.1147, 9: 0.1273, 10: 0.1169, 11: 0.1126, 12: 0.0944, 13: 0.0758, 



## Ejercicio 8

In [20]:
def ej8_poisson_sum(l, k):
    p = np.exp(-l); r = p
    for i in range(1, k+1): p *= l / i; r += p
    return r

def ej8_inv_method(l, k, n):
    def sim_inv_method(p, x):
        r = rnd.random()
        i = 0; prob = p[i]
        while r >= prob:
            if i+1 == len(p): return -1
            i += 1; prob += p[i]
        return x[i]

    denom = ej8_poisson_sum(l, k)
    prob = np.exp(-l)/denom; p = [prob]
    while len(p) < k+1: prob *= l / i; p.append(prob)
    x = [i for i in range(k+1)]

    assert len(x) == len(p) and len(x) == k+1

    p, x = zip(*sorted(zip(p, x), reverse=True))
    vals = []
    while len(vals) < n:
        v = sim_inv_method(p, x)
        if v != -1: vals.append(v)

    # Check if the distribution is correct
    c = dict()
    for v in vals: c[v] = c.get(v, 0) + 1
    for k in sorted(c.keys()): print(f"{k}: {c[k]/sum(c.values())}", end=", ")
    print()

    return sum(1 if v > 2 else 0 for v in vals) / len(vals)

def ej8_rejection_method(l, k, n):
    denom = ej8_poisson_sum(l, k)
    prob = np.exp(-l)/denom; p = [prob]
    while len(p) < k+1: prob *= l / i; p.append(prob)

    assert len(p) == k+1

    def nw_p(x): return p[x]
    def nw_q(x): return 1 / (k+1)
    def nw_Y(): return uniform_discrete_distribution(0, k)[0]
    c = nw_p(0) / nw_q(0) + 1e-4

    vals = rejection_method(nw_Y, nw_p, nw_q, c, n)

    # Check if the distribution is correct
    c = dict()
    for v in vals: c[v] = c.get(v, 0) + 1
    for k in sorted(c.keys()): print(f"{k}: {c[k]/sum(c.values())}", end=", ")
    print()

    return sum(1 if v > 2 else 0 for v in vals) / len(vals)

k = 10
l = 0.7
n = 10**4

print(f"EJERCICIO 8: Método de la transformada inversa")
v = ej8_inv_method(l, k, n)
print(f"    La probabilidad de que X > 2 es: {v}")
print()

print(f"EJERCICIO 8: Método de rechazo")
v = ej8_rejection_method(l, k, n)
print(f"    La probabilidad de que X > 2 es: {v}")
print()

EJERCICIO 8: Método de la transformada inversa
0: 0.8616, 1: 0.1209, 2: 0.0156, 3: 0.0015, 4: 0.0003, 5: 0.0001, 
    La probabilidad de que X > 2 es: 0.0019

EJERCICIO 8: Método de rechazo
0: 0.8622, 1: 0.1158, 2: 0.0182, 3: 0.0032, 4: 0.0006, 
    La probabilidad de que X > 2 es: 0.0038



## Ejercicio 9

In [21]:
def ej9_inv_method(p, n):
    vals = geometric_distribution(p, n)
    return sum(vals) / n

def ej9_simulation(p, n):
    vals = []
    for _ in range(n):
        vals.append(0)
        while True:
            vals[-1] += 1
            if bernoulli_distribution(p)[0] == 1: break
    return sum(vals) / n

p_list = [0.8, 0.2]
e_real = [1/p for p in p_list]
n = 10**4

print(f"EJERCICIO 9: Método de la transformada inversa")
for p in p_list:
    v = ej9_inv_method(p, n)
    print(f"    Para p = {p} la cantidad de lanzamientos esperados es: {v}, siendo el valor real: {e_real[p_list.index(p)]}")
print()

print(f"EJERCICIO 9: Método de simulación")
for p in p_list:
    v = ej9_simulation(p, n)
    print(f"    Para p = {p} la cantidad de lanzamientos esperados es: {v}, siendo el valor real: {e_real[p_list.index(p)]}")
print()

EJERCICIO 9: Método de la transformada inversa
    Para p = 0.8 la cantidad de lanzamientos esperados es: 1.2592, siendo el valor real: 1.25
    Para p = 0.2 la cantidad de lanzamientos esperados es: 5.0233, siendo el valor real: 5.0

EJERCICIO 9: Método de simulación
    Para p = 0.8 la cantidad de lanzamientos esperados es: 1.2555, siendo el valor real: 1.25
    Para p = 0.2 la cantidad de lanzamientos esperados es: 4.9374, siendo el valor real: 5.0



## Ejercicio 10

In [22]:
def ej10_inv_method():
    a = 1/2**2; b = 1/2*2**0/3**1
    nxtA = 1/2; nxtB = 2/3
    p = a+b; i = 1
    r = rnd.random()
    while r >= p:
        i += 1
        a *= nxtA; b *= nxtB
        p += a+b
    return i

def ej10_simulation(n):
    vals = [ej10_inv_method() for _ in range(n)]
    return sum(vals) / n

n = 10**3
print(f"EJERCICIO 10: Método de la transformada inversa")
v = ej10_simulation(n)
print(f"    La cantidad de lanzamientos esperados es: {v}")
print()

EJERCICIO 10: Método de la transformada inversa
    La cantidad de lanzamientos esperados es: 2.541



## Ejercicio 11

In [23]:
p = 0.2

print(f"EJERCICIO 11: Método de tasa de riesgo (p = {p})")
def P(x): return 0 if x == 0 else p*(1-p)**(x-1)
vals = risk_rate_method(P, 10**4)
c = dict()
for v in vals: c[v] = c.get(v, 0) + 1
print("     ", end="")
for k in sorted(c.keys()): print(f"{k}: {c[k]/sum(c.values())}", end=", ")
print("\n")

print(f"EJERCICIO 11: Comparación con la distribución geométrica (transformada inversa) con p = {p}")
vals = geometric_distribution(p, 10**4)
c = dict()
for v in vals: c[v] = c.get(v, 0) + 1
print("     ", end="")
for k in sorted(c.keys()): print(f"{k}: {c[k]/sum(c.values())}", end=", ")
print("\n")

print(f"EJERCICIO 11: Las primeras 20 tasas de riesgo son las siguientes para p = {p}")
i = 0; prob = P(i); s = 0
while i < 20:
    print(f"    {i}: {prob / (1 - s)}")
    i += 1; s += prob; prob = P(i)

EJERCICIO 11: Método de tasa de riesgo (p = 0.2)
     1: 0.2016, 2: 0.1545, 3: 0.1325, 4: 0.1009, 5: 0.0833, 6: 0.0705, 7: 0.0502, 8: 0.0393, 9: 0.0362, 10: 0.0245, 11: 0.0226, 12: 0.0171, 13: 0.0148, 14: 0.0096, 15: 0.0091, 16: 0.0054, 17: 0.0053, 18: 0.0038, 19: 0.0045, 20: 0.0029, 21: 0.0023, 22: 0.0016, 23: 0.0022, 24: 0.0007, 25: 0.0007, 26: 0.0008, 27: 0.0007, 28: 0.0003, 29: 0.0002, 30: 0.0003, 31: 0.0002, 32: 0.0003, 33: 0.0007, 34: 0.0001, 35: 0.0002, 39: 0.0001, 

EJERCICIO 11: Comparación con la distribución geométrica (transformada inversa) con p = 0.2
     1: 0.1949, 2: 0.1693, 3: 0.1276, 4: 0.1033, 5: 0.0793, 6: 0.0644, 7: 0.0489, 8: 0.0398, 9: 0.0337, 10: 0.0286, 11: 0.023, 12: 0.0173, 13: 0.0136, 14: 0.0108, 15: 0.0093, 16: 0.0065, 17: 0.0061, 18: 0.0057, 19: 0.0041, 20: 0.0024, 21: 0.0018, 22: 0.0024, 23: 0.0015, 24: 0.0011, 25: 0.0009, 26: 0.0014, 27: 0.0004, 28: 0.0006, 29: 0.0005, 30: 0.0003, 31: 0.0001, 33: 0.0001, 34: 0.0002, 42: 0.0001, 

EJERCICIO 11: Las primer

## Ejercicio 12

Para este ejercicio, tenemos que notar que la distribución que se simula en el algoritmo es $min(X, Y)$ donde $X\sim Geom(p_1)$ y $Y\sim Geom(p_2)$.

Si $X$ e $Y$ son independientes, como se muestra por la forma en la que se generan sus valores, entonces $min(X, Y)\sim Geom(1-(1-p_1)(1-p_2))$, por lo que podemos usar eso para usar un único número aleatorio.

Luego, nos queda entonces lo siguiente:

In [24]:
def ej12_original(p1, p2, n):
    return [min(geometric_distribution(p1)[0], geometric_distribution(p2)[0]) for _ in range(n)]

def ej12_optimized(p1, p2, n):
    return geometric_distribution(1-(1-p1)*(1-p2), n)

p1 = 0.05; p2 = 0.2
n = 10**4

print(f"EJERCICIO 12: Método original")
vals = ej12_original(p1, p2, n)
c = dict()
for v in vals: c[v] = c.get(v, 0) + 1
print("     ", end="")
for k in sorted(c.keys()): print(f"{k}: {c[k]/sum(c.values())}", end=", ")
print("\n")

print(f"EJERCICIO 12: Método optimizado")
vals = ej12_optimized(p1, p2, n)
c = dict()
for v in vals: c[v] = c.get(v, 0) + 1
print("     ", end="")
for k in sorted(c.keys()): print(f"{k}: {c[k]/sum(c.values())}", end=", ")
print("\n")

EJERCICIO 12: Método original
     1: 0.2418, 2: 0.1833, 3: 0.1385, 4: 0.1053, 5: 0.0769, 6: 0.0649, 7: 0.0438, 8: 0.034, 9: 0.0253, 10: 0.023, 11: 0.0149, 12: 0.0118, 13: 0.009, 14: 0.007, 15: 0.0048, 16: 0.0026, 17: 0.0028, 18: 0.0025, 19: 0.0021, 20: 0.0019, 21: 0.0012, 22: 0.0007, 23: 0.0004, 24: 0.0003, 25: 0.0004, 26: 0.0004, 29: 0.0003, 30: 0.0001, 

EJERCICIO 12: Método optimizado
     1: 0.2431, 2: 0.1794, 3: 0.1331, 4: 0.1084, 5: 0.081, 6: 0.0653, 7: 0.0471, 8: 0.0353, 9: 0.0295, 10: 0.0188, 11: 0.0144, 12: 0.0102, 13: 0.0079, 14: 0.0069, 15: 0.0045, 16: 0.0036, 17: 0.0018, 18: 0.0016, 19: 0.0021, 20: 0.0011, 21: 0.0014, 22: 0.0009, 23: 0.0006, 24: 0.0006, 25: 0.0003, 26: 0.0002, 27: 0.0002, 29: 0.0002, 30: 0.0001, 32: 0.0002, 33: 0.0002, 

